#### Business Analytics FHDW 2025
# Naives Bayes-Verfahren
## am Beispiel der Vorhersage verzögerter Flüge

In der engen Taktung internationaler Lufträume und der lokalen Organisation von teuren und komplexen Flughäfen ist der Umgang mit Verspätungen ein kritisches Thema. Es betrifft Kosten mit engen und teuren Start- und Lande-Slots, aber insbesondere auch die Sicherheit, da die Bewegungen im Luftraum nicht einfach angehalten werden können, um auf verspätete Flugzeuge zu warten und zeitliche/örtliche Abstände einzuhalten sind.

Wenn wir auf Basis von Daten also Verspätungen von Flügen zuverlässig vorhersagen können, schützt das Menschen und spart viel Geld. Wir betrachten den Datensatz *FlightDelays* mit Daten amerikanischer Flughäfen und -gesellschaften. Daraus nutzen wir als Prädiktoren
- Wochentag als 1 = Montag,..., 7 = Sonntag,
- geplante Abflugzeit in 18 Zeitintervallen zwischen 6:00 am und 10:00 pm,
- Herkunft als drei Flughafen-Codes DCA, IAD, BWI,
- Zielflughäfen JFK, LGA, EWR,
- Fluggesellschaften CO, DH, DL, MQ, OH, RU, UA, US.

Zielvariable ist der *Status* als pünktlich oder verspätet.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart

delays_df = pd.read_csv('./Daten/FlightDelays.csv')
delays_df.head

Gemäß der für Bayes definierten kategorischen Prädiktoren wandeln wir zunächst die Daten um und erzeugen passende Dummy-Variablen.

In [ ]:
delays_df.DAY_WEEK = delays_df.DAY_WEEK.astype('category')
delays_df['Flight Status'] = delays_df['Flight Status'].astype('category')

delays_df.CRS_DEP_TIME = [round(t/100) for t in delays_df.CRS_DEP_TIME]
delays_df.CRS_DEP_TIME = delays_df.CRS_DEP_TIME.astype('category')

predictors = ['DAY_WEEK', 'CRS_DEP_TIME', 'ORIGIN', 'DEST', 'CARRIER']
outcome = 'Flight Status'
X = pd.get_dummies(delays_df[predictors])
y = delays_df[outcome].astype('category')
classes = list(y.cat.categories)
X

Dann teilen wir den modifizierten Datensatz in Trainings- und Validierungsdaten auf und führen den naiven Bayes mit `MultinomialNB` durch. Der Parameter `alpha` dient hier der (Laplace-)Glättung von Variablen, die in unseren Trainingsdaten nur 0-Werte besitzen. *Alpha* > 0 vermeidet ein Overfitting des Modells, so dass aus einer Stichprobe, in der eine bestimmte Kategorie fehlt, nicht deren generelle Nichtexistenz abgeleitet wird. 

Wir merken uns in Variablen sowohl die resultierenden Wahrscheinlichkeiten (`predict_proba`) als auch die Klassifizierungen (`predict`).

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=1)

delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

pred_prob_train = delays_nb.predict_proba(X_train)
pred_prob_valid = delays_nb.predict_proba(X_valid)

y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)

Statt nun einfach die Ergebnisse zu verwenden, generieren wir uns Pivot-Tabellen über die von den einzelnen Prädiktoren bedingten Wahrscheinlichkeiten.

In [ ]:
train_df, valid_df = train_test_split(delays_df, test_size=0.4, random_state=1)
pd.set_option('display.precision', 4)
print(train_df['Flight Status'].value_counts() / len(train_df))
print()
for predictor in predictors:
    tmp_df = train_df[['Flight Status', predictor]]
    freq_table = tmp_df.pivot_table(index='Flight Status', columns=predictor, aggfunc=len, observed=False)
    prop_table = freq_table.apply(lambda x: x/sum(x), axis=1)
    print(prop_table)
    print()
    
pd.reset_option('display.precision')

Um einen neuen Flug zu klassifizieren, berechnen wir die Wahrscheinlichkeiten für Verspätung und Pünktlichkeit. Dazu brauchen wir nur die Zähler der Bayes-Formel zu vergleichen, da beide Werte den gleichen Nenner besitzen.

Um z. B. einen Delta-Flug von DCA nach LGA zu klassifizieren, der sonntags zwischen 10.00 und 11.00 morgens abfliegt, berechnen wir die Zähler aus den Werten der Pivot-Tabelle oben:

$P(delayed|Carrier=DL, Day Week=7, Dep Time=10, Dest=LGA, Origin=DCA) \\ \propto (0.1977)(0.0958)(0.1609)(0.0307)(0.4215)(0.5211) = 0.000021$

$P(ontime|Carrier=DL, Day Week=7, Dep Time=10, Dest=LGA, Origin=DCA) \\ \propto (0.8023)(0.2040)(0.1048)(0.0519)(0.5779)(0.6478) = 0.00033$

$\propto$ bedeutet hier "ist proportional zu", da wir zunächst ja nur die Zähler ermittelt haben (es sieht auf den ersten Blick aus wie ein $\alpha$, ist aber keins).

Die Werte zeigen, dass die Pünktlichkeit des betrachteten Fluges wahrscheinlicher ist, als die Verspätung. Wichtig ist hier, zur Kenntnis zu nehmen, dass ein Datenpunkt mit genau dieser Kombination von Prädiktorwerten nicht in den Trainingsdaten vorkommt. Wir müssen also den naiven Bayes nutzen. Die tatsächlichen bedingten Wahrscheinlichkeiten können wir nun auch ermitteln:

$P(delayed|Carrier=DL, Day Week=7, Dep Time=10, Dest=LGA, Origin=DCA) = \frac{0.000021}{0.000021+0.00033} = 0.058$ 

$P(ontime|Carrier=DL, Day Week=7, Dep Time=10, Dest=LGA, Origin=DCA) = \frac{0.00033}{0.000021+0.00033} = 0.942$ 

Die ausführliche Darstellung soll deutlich machen, wie sich - trotz der üblichen Instanzierung eines Modells und der uniformen Aufrufe von `predict` - das Modell und seine Berechnungen bei Bayes von anderen unterscheidet. Natürlich nutzen wir "in der Realität" bzw. in der Praxis die Funktionen der Bibliotheken, um die Wahrscheinlichkeiten für die interessanten Datensätze aus den Trainings- oder Validierungsdaten, oder für einen neuen Datenpunkt zu bestimmen. Für den Beispieldatenpunkt von oben finden wir in den Validierungsdaten die gleichen Wahrscheinlichkeiten, wie zuvor händisch berechnet:

In [ ]:
search_df = pd.concat([pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred}),
                       pd.DataFrame(pred_prob_valid, index=y_valid.index)], axis=1)

search_mask = ((X_valid.CARRIER_DL == 1) & (X_valid.DAY_WEEK_7 == 1) & (X_valid.CRS_DEP_TIME_10 == 1)
               & (X_valid.DEST_LGA == 1) & (X_valid.ORIGIN_DCA == 1))

search_df[search_mask]

Zur Einschätzung der prädiktiven Performance des naiven Bayes-Klassifizierers lassen wir uns die Konfusionsmatrix angeben.

In [ ]:
classificationSummary(y_train, y_train_pred, class_names=classes)
print()
classificationSummary(y_valid, y_valid_pred, class_names=classes)

Sowohl für die Trainings- als auch die Validierungsdaten liegt die Genauigkeit bei knapp 80%. Hätten wir einfach alle Datensätze als pünktlich klassifiziert, wäre die Qualität nicht viel schlechter gewesen. Ein *Gains Chart* zeigt allerdings, dass der naive Bayes die verspäteten Flüge effektiv einfängt, wenn Ranking berücksichtigt wird:

In [ ]:
gal_df = pd.DataFrame({'actual': 1-y_valid.cat.codes, 'prob': pred_prob_valid[:, 0]})
gal_df = gal_df.sort_values(by=['prob'], ascending=False).reset_index(drop=True)

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
gainsChart(gal_df.actual, ax=ax)

## Aufgabe

Die reine Nutzung von `MultinomialNB` ist einfach. Daher schauen wir uns die unterschiedlichen Anwendungen des Satzes von Bayes am Beispiel des Datensatzes *UniversalBank.csv* noch einmal genauer an. Teilen Sie den Datensatz dazu in 60% Trainingsdaten und 40% Validierungsdaten auf.

1. Generieren Sie aus den Trainingsdaten eine *Pivot-Tabelle* mit *Online als Spaltenvariable*, *CreditCard als Zeilenvariable* und *PersonalLoan als sekundärer Zeilenvariable*. Nutzen Sie dazu die Dataframe-Methoden `melt` und `pivot_table`.

2. Bestimmen Sie aus der Pivot-Tabelle die Wahrscheinlichkeit, dass eine Kundin, die eine Kreditkarte besitzt und Online Banking nutzt, ein Darlehen in Anspruch nimmt (das ist die bedingte Wahrscheinlichkeit $P(Loan=1|CC=1,Online=1)$ eines Darlehens unter den Voraussetzungen Kreditkarte und Online).

3. Generieren Sie zwei weitere Pivot-Tabellen aus den Trainingsdaten: Eine mit Spaltenvariable *Online*, eine mit Spaltenvariable *CreditCard*, beide mit *PersonalLoan* als Zeilenvariable.

4. Bestimmen Sie folgende Wahrscheinlichkeiten aus den Werten der Pivot-Tabellen (CC = CreditCard):<br>
$P(CC=1|Loan=1)$ <br>
$P(Online=1|Loan=1)$<br>
$P(Loan=1)$<br>
$P(CC=1|Loan=0)$<br>
$P(Online=1|Loan=0)$<br>
$P(Loan=0)$<br>

5. Ermitteln Sie aus den Werten oben die Naive-Bayes-Wahrscheinlichkeit $P_{nb}(Loan=1|CC=1,Online=1)$. Vergleichen Sie diese mit dem Wert von oben. Welches ist der genauere Wert?

6. Generieren Sie für die Trainingsdaten ein Modell mit `MultinomialNB` und ermitteln Sie damit die Wahrscheinlichkeit $P(Loan=1|CC=1,Online=1)$.